<a href="https://colab.research.google.com/github/josueyoon153/Trabajos_diplo_2021_josueyoon/blob/main/Trabajo_Final_CNN_Style_Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Style Transfer

<img src="https://i0.wp.com/chelseatroy.com/wp-content/uploads/2018/12/neural_style_transfer.png?resize=768%2C311&ssl=1">

La idea de este trabajo final es reproducir el siguiente paper:

https://arxiv.org/pdf/1508.06576.pdf

El objetivo es transferir el estilo de una imagen dada a otra imagen distinta. 

Como hemos visto en clase, las primeras capas de una red convolucional se activan ante la presencia de ciertos patrones vinculados a detalles muy pequeños.

A medida que avanzamos en las distintas capas de una red neuronal convolucional, los filtros se van activando a medida que detectan patrones de formas cada vez mas complejos.

Lo que propone este paper es asignarle a la activación de las primeras capas de una red neuronal convolucional (por ejemplo VGG19) la definición del estilo y a la activación de las últimas capas de la red neuronal convolucional, la definición del contenido.

La idea de este paper es, a partir de dos imágenes (una que aporte el estilo y otra que aporte el contenido) analizar cómo es la activación de las primeras capas para la imagen que aporta el estilo y cómo es la activación de las últimas capas de la red convolucional para la imagen que aporta el contenido. A partir de esto se intentará sintetizar una imagen que active los filtros de las primeras capas que se activaron con la imagen que aporta el estilo y los filtros de las últimas capas que se activaron con la imagen que aporta el contenido.

A este procedimiento se lo denomina neural style transfer.

# En este trabajo se deberá leer el paper mencionado y en base a ello, entender la implementación que se muestra a continuación y contestar preguntas sobre la misma.

# Una metodología posible es hacer una lectura rápida del paper (aunque esto signifique no entender algunos detalles del mismo) y luego ir analizando el código y respondiendo las preguntas. A medida que se planteen las preguntas, volviendo a leer secciones específicas del paper terminará de entender los detalles que pudieran haber quedado pendientes.

Lo primero que haremos es cargar dos imágenes, una que aporte el estilo y otra que aporte el contenido. A tal fin utilizaremos imágenes disponibles en la web.

In [1]:
# Imagen para estilo
!wget https://upload.wikimedia.org/wikipedia/commons/5/52/La_noche_estrellada1.jpg

# Imagen para contenido
!wget https://upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Neckarfront_T%C3%BCbingen_Mai_2017.jpg/775px-Neckarfront_T%C3%BCbingen_Mai_2017.jpg

# Creamos el directorio para los archivos de salida
!mkdir /content/output

In [2]:
from keras.preprocessing.image import load_img, save_img, img_to_array
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time
import argparse

from keras.applications import vgg19
from keras import backend as K
from pathlib import Path



import tensorflow as tf
tf.compat.v1.disable_eager_execution()

In [3]:
#IMAGEN BASE ELEGIDO PARA EL ULTIMO PUNTO
from google.colab import files
base_image = files.upload()

Saving IMG_1763.JPG to IMG_1763.JPG


In [4]:
#IMAGEN ESTILO ELEGIDO PARA EL ULTIMO PUNTO
style_reference_image = files.upload()

Saving Manifestacion-Berni-300-1024x734.jpeg to Manifestacion-Berni-300-1024x734.jpeg


In [3]:
# Definimos las imagenes que vamos a utilizar, y el directorio de salida

base_image_path = Path("/content/775px-Neckarfront_Tübingen_Mai_2017.jpg")
style_reference_image_path = Path("/content/La_noche_estrellada1.jpg")
result_prefix = Path("/content/output")
iterations = 100

In [5]:
# Imagenes del
base_image_path = Path("/content/IMG_1763.JPG")
style_reference_image_path = Path("/content/Manifestacion-Berni-300-1024x734.jpeg")
result_prefix = Path("/content/output")
iterations = 100

In [4]:
base_image_path

PosixPath('/content/775px-Neckarfront_Tübingen_Mai_2017.jpg')

# 1) En base a lo visto en el paper ¿Qué significan los parámetros definidos en la siguiente celda?

Respuesta: La idea central de este paper es separar el CONTENIDO y el ESTILO de una imagen utilizando una red convolucional. De esta forma poder manipular independientemente estos dos aspectos para luego poder obtener una nueva imagen combinando el contenido y el estilo de dos imagenes diferentes. Lo que el paper muestra detalladamente es que podemos regular el énfasis de la reconstrucción del estilo y del contenido. Sabiendo esto la variable "style_weight" hace referencia al peso que le vamos a estar dando al estilo y la variable "content_weight" hace referencia al peso que le vamos a estar danto al contenido de la imagen. Si el peso del estilo es más alto, intuitivamente podemos imaginarnos una imagen que predomine más el toque artístico donde captaríamos menos el contenido de la fotografía. Por el contrario si el peso del contenido es más alto identificaríamos claramente los rasgos de la fotografía y menos el toque artístico. Estos pesos van a estar ponderando la loss total que esta compuesta por la content_loss y la style_loss



In [6]:
#peso original de la notebook
total_variation_weight = 0.1
style_weight = 10
content_weight = 1

In [4]:
#mucho mayor peso en el estilo
total_variation_weight = 0.1
style_weight = 10000
content_weight = 1

In [4]:
#mucho mayor peso en el contenido
total_variation_weight = 0.1
style_weight = 1
content_weight = 100

In [7]:
# Definimos el tamaño de las imágenes a utilizar
width, height = load_img(base_image_path).size
img_nrows = 400
img_ncols = int(width * img_nrows / height)

# 2) Explicar qué hace la siguiente celda. En especial las últimas dos líneas de la función antes del return. ¿Por qué?

Ayuda: https://keras.io/applications/

**Respuesta**: 

En las siguientes líneas de código lo que se define es un método que nos va a permitir cargar y preprocesar las imágenes de una forma rápida y eficiente utilizando la arquitectura VGG19 de preprocesamiento, una red de clasificación de imágenes previamente entrenada.
Lo que se hace con np.expand_dims es expandir el shape del array indicando axis=0 lo que va a aumentar en una unidad la dimensión de la imagen cargada. Si vamos paso a paso en la segunda línea de código al pasar la imagen en array voy a tener el shape que tiene una imagen (a color en este caso que es [row size, col size, 3]). Sin embargo como KERAS trabaja con batch de imágenes necesitamos agregar una dimensión adicional lo cual la primera dimensión es utilizado por el número de samples (cant de imágenes) que tengo: (sample, row size, col size, 3)
El preprocess_input es para adecuar la imagen a un formato que necesita el modelo. Esto es porque el modelo que usamos fue entrenado en distintos dataset por lo que el shape sigue siendo (sample, row size, col size, 3). 
El preprocess_input resta la media de los canales RGB de la dataset del imagenet. En otras palabras estamos normalizando los canales de los colores acorde a qué dataset fue usado en el entrenamiento de las redes anteriormente. 

Let's create methods that will allow us to load and preprocess our images easily. We perform the same preprocessing process as are expected according to the VGG training process. VGG networks are trained on image with each channel normalized by mean = [103.939, 116.779, 123.68]and with channels BGR.

In [8]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_nrows, img_ncols))#aca simplemente cargo la imagen que me interesa analizar con el tamaño ya definido
    img = img_to_array(img)#transformo la img de forma matricial donde le shape es (400{img_nrows}, 517{img_ncols}, 3{rgb})
    img = np.expand_dims(img, axis=0)#le agrego una dimension adicional donde ahora el shape es  (1, 400, 517, 3)
    img = vgg19.preprocess_input(img)#The images are converted from RGB to BGR, then each color channel is zero-centered with respect to the ImageNet dataset, without scaling.
    return img

# 3) Habiendo comprendido lo que hace la celda anterior, explique de manera muy concisa qué hace la siguiente celda. ¿Qué relación tiene con la celda anterior?

Respuesta: lo que hace esta celda es el camino inverso a lo que hace el preprocess_input(). Antes la idea era convertir las imágenes de RGB a BGR centrando a cero cada canal de color con respecto al dataset del imagenet. Esto se hacía restando la media de los canales RGB es decir, normalizar los canales de los colores acorde a qué dataset fue usado en el entrenamiento de las redes anteriormente. Sin embargo acá podemos observar que vuelve a hacer un reshape en 3 dimensiones y en lugar de restar la media hace una suma de las media. Aca estaríamos pasando de BGR a RGB. La relación es que es exactamente el camino inverso.

In [9]:
def deprocess_image(x):
    x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [10]:
# get tensor representations of our images
# K.variable convierte un numpy array en un tensor, para 
base_image = K.variable(preprocess_image(base_image_path))
style_reference_image = K.variable(preprocess_image(style_reference_image_path))

In [11]:
combination_image = K.placeholder((1, img_nrows, img_ncols, 3))

Aclaración:

La siguiente celda sirve para procesar las tres imagenes (contenido, estilo y salida) en un solo batch.

In [12]:
# combine the 3 images into a single Keras tensor
input_tensor = K.concatenate([base_image,
                              style_reference_image,
                              combination_image], axis=0)

In [13]:
# build the VGG19 network with our 3 images as input
# the model will be loaded with pre-trained ImageNet weights
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet', include_top=False)
print('Model loaded.')

# get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

80150528/80134624 [==============================] - 1s 0us/step
Model loaded.


# 4) En la siguientes celdas:

- ¿Qué es la matriz de Gram?¿Para qué se usa?

Respuesta: Un estilo de una imagen es la correlación entre formas, colores y distribuciones de las mismas. Entonces podemos usar la red para extraer el estilo a través de las feature matrices. Para esto se utiliza la matriz de Gram que representa un estilo de una imagen computando la correlación entre diferentes features matrices las cuales ayuda a capturar el estilo de una imagen. (similitud entre filtros)

Es importante saber lo que significa el producto punto entre dos vectores para poder entender mejor lo que hace una matriz de Gram: Teóricamente sabemos que el producto punto de dos vectores es la suma de los productos de las coordenadas repectivas. También lo entendemos como la longitud del vector "a" que va en la misma dirección que el vector b multiplicado por la longitud del vector "b". Intuitivamente muestra cuán similares son dos vectores entre ellos. Entonces imaginemos dos flattened feature vectors de un feature map de una convolucion. El producto punto de estos nos daría la información sobre la relación que tienen entre ellos. Mientras más chico sea el número del producto punto son más diferentes los features capurados y cuan más grande sea la misma habría más correlación entre los features. Esto nos da información sobre el estilo y la textura pero cero información sobre la estructura espacial debido a que ya se aplicó el flatten como un vector.

Al computar el producto punto de todos los feature vectors (flattened de una feature map de una convolucional), tenemos como resultado una matriz de GRAM.

- ¿Por qué se permutan las dimensiones de x?

Las dimensiones de x se permutan porque lo que necesitamos hacer es que las filas sean los canales y las columnas las filas de cada canal que luego son flattened.

In [14]:
def gram_matrix(x):
    #hange height,width,depth to depth, height, width, it could be 2,1,0 too
    ## We want each row to be a channel, and the columns to be flattened x,y locations
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))#aca estoy haciendo el producto punto del 
    #feature que es el vector flatten por la traspuesta de la misma
    return gram

# 5) Losses:

Explicar qué mide cada una de las losses en las siguientes tres celdas.

Rta:

Content Loss: El objetivo del content loss es calcular cuan símiles o disímiles son los contenidos entre la imagen generada y la imagen de entrada que fue utilizada como content image.
Capas más avanzadas de una convolucional contienen información de las macro estructuras. Se conoce que la CNN produce distintos features maps en capas mas avanzadas siendo estas activadas en presencia de diferentes objetos. Esto nos ayuda a deducir que las imágenes con los mismos contenidos deberían tambien tener activaciones similares en los higher layers.
Siendo "Combination" la imagen output generada y "base" la imagen input de contenido, podemos ver que se eleva al cuadrado la resta entre los dos. El objetivo de esto es minimizar el content loss lo que significaría que la imagen generada sea muy parecida en contenido de la imagen input. Mientras más se parezca el contenido de la imagen output con la content image, el cuadrado de la resta va a ser menor.

Style Loss: El concepto de este style loss es muy parecido al content loss solo que en lugar de tomar la distancia entre la imagen generada como output y la imagen input de contenido voy a tomar la imagen input que utilicé para captar el estilo. Entonces en este caso estamos tratando de saber cuan lejos/distante esta el estilo de nuestra imagen generada en comparación al style image.
En el caso anterior comparamos los outputs de las capas intermedias de la convolución. Sin embargo no podemos hacer lo mismo para el style loss y necesitamos lo que es la gram matrix. Aca nosotros comparamos la diferencian entre las matrices de gram de una capa especifica para la imagen input de estilo y la imagen generada.
En términos muy simples tendríamos:
style loss = Generated image style - style image
Para obtener el estilo tanto de la imagen generada como la imagen input de estilo necesitamos calcular la gram matrix de cada imagen. Por este motivo podemos ver que a "S" y "C" se le calcula la función gram matrix que definimos mas arriba.


Total Variation Loss: Hemos dicho que el content loss calcula la diferencia que hay en el contenido de la imagen generada con el content input image. y el style loss calcula la diferencia de estilo entre la imagen generada con el style input image. Entonces con esto podemos calcular la total loss. simple mente significa content loss + style loss. 

In [16]:
def style_loss(style, combination):
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

In [17]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))


In [18]:
def total_variation_loss(x):
    assert K.ndim(x) == 4
    a = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
    b = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))


In [19]:
# Armamos la loss total
loss = K.variable(0.0)
layer_features = outputs_dict['block5_conv2']
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss = loss + content_weight * content_loss(base_image_features,
                                            combination_features)

feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']
for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :] 
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss = loss + (style_weight / len(feature_layers)) * sl
loss = loss + total_variation_weight * total_variation_loss(combination_image)

In [19]:
print(loss)

Tensor("add_7:0", shape=(), dtype=float32)


In [20]:

#import tensorflow as tf
#tf.compat.v1.disable_eager_execution()


#with tf.GradientTape() as gtape:
#  grads = gtape.gradient(loss, combination_image)
#  grads = tape.gradient(loss, w)
#grads = K.gradients(loss, combination_image)
#grads = tf.GradientTape(loss, combination_image)
grads = K.gradients(loss, combination_image)

outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([combination_image], outputs)

# 6) Explique el propósito de las siguientes tres celdas. ¿Qué hace la función fmin_l_bfgs_b? ¿En qué se diferencia con la implementación del paper? ¿Se puede utilizar alguna alternativa?

Respuesta:

eval_loss_and_grads: es para obtener el valor de la función de coste y los valores de los gradientes de cada imagen modificada que le pasamos como argumento.
El método gradients de la celda anterior nos proporciona los gradientes de la imagen que minimizan la funcion de coste. Con esta información, después generamos una imagen modificada que se acerca a nuestro objetivo respecto a la imagen actual. Cada imagen modificada se va a pasar como argumento a eval_loss_and_grads() y así vamos obteniendo sucesivas imágenes que cada vez cumplen más con nuestro objetivo que es minimizar la función de coste es decir, que se parezcan a la imagen base pero con el estilo de la style image. En el punto 7 del trabajo se implementa esta iteración con un bucle donde se obtiene repetitivamente una imagen modificada/mejorada hacia nuestro objetivo.

La funcion fmin_l_bfgs_b es un optimizador con una limitación donde requiere el loss y la gradiente de forma separada. Debido a que computar estos dos de forma independiente es extremadamente ineficiente en este caso se implementa un evaluator class que computa la loss y los valores de la grandiente de una sola vez.
Este optimizador busca minimizar una función utilizando el algoritmo L-BFGS-B. Es una optimización que se realiza utilizando la derivada de segundo orden de una función objetivo. El algoritmo BFGS es uno de los algoritmos de segundo orden más utilizados para la optimización numérica y se usa comúnmente para adaptarse a los algoritmos de aprendizaje automático.

In [21]:
def eval_loss_and_grads(x):
    x = x.reshape((1, img_nrows, img_ncols, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

# this Evaluator class makes it possible
# to compute loss and gradients in one pass
# while retrieving them via two separate functions,
# "loss" and "grads". This is done because scipy.optimize
# requires separate functions for loss and gradients,
# but computing them separately would be inefficient.

In [22]:
class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

# 7) Ejecute la siguiente celda y observe las imágenes de salida en cada iteración.

In [23]:
evaluator = Evaluator()

# run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss
x = preprocess_image(base_image_path)

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # save current generated image
    img = deprocess_image(x.copy())
    fname = result_prefix / ('output_at_iteration_%d.png' % i)
    save_img(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0
Current loss value: 20288444000.0
Image saved as /content/output/output_at_iteration_0.png
Iteration 0 completed in 18s
Start of iteration 1
Current loss value: 9433432000.0
Image saved as /content/output/output_at_iteration_1.png
Iteration 1 completed in 6s
Start of iteration 2
Current loss value: 6965411300.0
Image saved as /content/output/output_at_iteration_2.png
Iteration 2 completed in 6s
Start of iteration 3
Current loss value: 5599468500.0
Image saved as /content/output/output_at_iteration_3.png
Iteration 3 completed in 6s
Start of iteration 4
Current loss value: 4859497500.0
Image saved as /content/output/output_at_iteration_4.png
Iteration 4 completed in 6s
Start of iteration 5
Current loss value: 4383176700.0
Image saved as /content/output/output_at_iteration_5.png
Iteration 5 completed in 6s
Start of iteration 6
Current loss value: 4023466200.0
Image saved as /content/output/output_at_iteration_6.png
Iteration 6 completed in 6s
Start of iteration 7
Curr

# 8) Generar imágenes para distintas combinaciones de pesos de las losses. Explicar las diferencias. (Adjuntar las imágenes generadas como archivos separados.)

Respuesta: Como default en la notebook tenemos content_weight=1 y style_weight=10 dándole un poco mas de peso al estilo para que el énfasis esté en el style transfer asi como se llama la metodología. Con estos pesos podemos observar que a medida que va iterando obtenemos como output imágenes que conserva el contenido pero que claramente tiene el estilo del style image. Lo que hacemos con estos pesos es influenciar la loss del contenido y de estilo para que se pueda ir jugando con el énfasis puesto en el contenido o en el estilo. Viendo las imágenes generadas con este peso podemos decir que tiene un ratio donde a simple vista el énfasis está distribuído "equitativamente" (sabiendo que en realidad estamos ponderando más el estilo que el contenido) ya que conserva bien el contenido y las formas del contenido tanto como el estilo que toma de la obra de Van Gogh. Quizas en las primeras no se nota mucho la transferencia del estilo pero ya en las ultimas iteraciones podemos ver claramente el traspaso del estilo.

Luego se hicieron 2 pruebas mas: una que pondere exageradamente mas el estilo y otro el contenido.

Para el primer caso mencionado con mayor peso en el estilo los parámetros utilizados fueron:

- style_weight = 10000
- content_weight = 1

Con estos parámetros se observó claramente la distorsión de las formas de los elementos que se ven en la imagen original como por ejemplo las casas. Al tener un peso muy grande en minimizar la loss del estilo las líneas y las formas son bastante más invadidas por el estilo.

y para el caso que se dió mayor importancia al contenido los parámetros fueron:

- style_weight: 1
- content_weight: 100

Con estos parámetros se observó una clara diferencia con los casos anteriores. Aunque la iteracion iba avanzando, el estilo de la obra de Van Gogh no predominó en el output de la imagen. Las formas de las casas y de los árboles se vieron definitivamente más nítidos mas cercanos a la foto real. 

# 9) Cambiar las imágenes de contenido y estilo por unas elegidas por usted. Adjuntar el resultado.

Respuesta: Como content_image elegí una fotografía tomada por mí en el medio de la ruta cuando estuve viajando por el norte de nuestro país (Cafayate). 
Como style image elegí la obra "Desocupados" de Berni (Artista argentino) debido a que casi el 80% de la imagen está cubierta por rostros muy típico del estilo de Berni. Me dió curiosidad de si el algoritmo tomaría el rostro como un estilo como para plasmarlo en un paisaje.